# VLM/MLLM -D5/D6 Single

In [1]:
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from keras.callbacks import EarlyStopping
from keras.layers import (Activation, Attention, Bidirectional, Concatenate, Conv1D,
                           Dense, Dropout, Embedding, Flatten, GlobalMaxPooling1D,
                           Input, Layer, LSTM, MaxPooling1D, Multiply, Permute,
                           RepeatVector, Reshape, SpatialDropout1D, TimeDistributed)
from keras.models import Model
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from sklearn.ensemble import ExtraTreesClassifier
import sys
sys.path.append('./cs16')
import cs16.prep as prep16
import cs16.plot as plot16
import cs16.build as build16
imagesize = 64

import pandas as pd

#data_type = input("Enter '1. single' for single dataset or '2. multi' for multiple dataset: ")
data_type = '1'
if data_type == '1':
    file_path = 'single.txt'
    folder_path = './data/MVSA/single/'
elif data_type == '2':
    file_path = 'multi.txt'
    folder_path = './data/MVSA/multiple/'
else:
    print("Invalid input. Please enter either 'single' or 'multi'.")
    exit()

df = pd.read_csv(file_path, index_col=None, encoding='ISO-8859-1')

Enter '1. single' for single dataset or '2. multi' for multiple dataset: 1


In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import matplotlib.pyplot as plt
# Download stopwords and punkt tokenizer if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Define a function to preprocess text
def nlp_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove URLs using regex
    text = re.sub(r'http\S+', '', text)
    
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize text into individual words
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Stem words using Porter Stemmer
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    # Join words back into a single string
    text = ' '.join(words)
    
    return text

# Apply the preprocess_text function to the 'tweet' column of the dataframe
df['tweet'] = df['tweet'].apply(nlp_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ausco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ausco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
imgsize=96
X_text, y_text = prep16.preprocess_text(df)
X_train_text, X_val_text, X_test_text, \
y_train_text, y_val_text, y_test_text = prep16.split_data(X_text, y_text, random_state=42)

X_polar, y_polar = prep16.preprocess_text(df,label = 'polarity')
X_train_polar, X_val_polar, X_test_polar, \
y_train_polar, y_val_polar, y_test_polar = prep16.split_data(X_polar, y_polar, random_state=42)

image_data_s, image_label_s = prep16.preprocess_images(df, folder_path, imagesize =imgsize)
y_s = to_categorical(image_label_s, num_classes=3)

X_train_image, X_val_image, X_test_image, \
y_train_image, y_val_image, y_test_image= prep16.split_data(image_data_s, y_s, random_state=42)

y_train = to_categorical(y_train_polar, num_classes=3)
y_val =to_categorical(y_val_polar, num_classes=3)
y_test =to_categorical(y_test_polar, num_classes=3)
file_path

'single.txt'

In [88]:
image_data_s.shape, X_train_image.shape, y_s.shape

((4869, 96, 96, 3), TensorShape([3895, 96, 96, 3]), (4869, 3))

# 1. EARLY FUSION TinyCLIP

In [4]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalAveragePooling2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from transformers import AutoTokenizer, TFCLIPModel
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

# ------------------------------------------------------------
# Timer Start
start_time = time.time()

# ------------------------------------------------------------
# TinyCLIP/CLIP TEXT
# ------------------------------------------------------------
text_model_name = "wkcn/TinyCLIP-ViT-61M-32-Text-29M-LAION400M"

tokenizer = AutoTokenizer.from_pretrained(text_model_name)
text_encoder = TFCLIPModel.from_pretrained(text_model_name)
text_encoder.trainable = False

def get_text_features(texts, processor, model, maxlen=77):
    encodings = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=maxlen,
        return_tensors="tf"
    )
    features = model.get_text_features(
        input_ids=encodings["input_ids"]
    )
    return features.numpy()

# Text 
X_train_text = [str(t) for t in X_train_text]
X_val_text = [str(t) for t in X_val_text]
X_test_text = [str(t) for t in X_test_text]

# ------------------------------------------------------------
# MobileNetV2 IMAGE 
# ------------------------------------------------------------
image_input = Input(shape=(imgsize, imgsize, 3), name='image_input')
mobilenet = tf.keras.applications.MobileNetV2(include_top=False, 
                                            weights='imagenet', 
                                            input_shape=(imgsize, imgsize, 3))
mobilenet.trainable = False
image_features = mobilenet(image_input)

#text
text_input = Input(shape=(512,), name='text_input')  # CLIP 512 maxlength
text_embedding = Dense(128, activation='relu')(text_input)   
text_embedding = Reshape((1, 1, 128))(text_embedding)  #Preparing for fusion

# image
# Expand text features -> to the size of image features
h, w = image_features.shape[1], image_features.shape[2]
text_features_expanded = tf.keras.layers.UpSampling2D(size=(h, w))(text_embedding)

# Early Fusion: Feature Level 
fusion = Concatenate(axis=-1)([image_features, text_features_expanded])

# reduce dim
x = GlobalAveragePooling2D()(fusion)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.3)(x)
output = Dense(3, activation='softmax')(x)

model = Model(inputs=[text_input, image_input], outputs=output)
#Reduce Overfitting
opti = tf.keras.optimizers.AdamW(learning_rate=1e-5, clipnorm=1.0)
model.compile(loss='categorical_crossentropy', optimizer=opti, metrics=['accuracy'])

# ------------------------------------------------------------
# to vector for fusion
X_train_text_feat = get_text_features(X_train_text, tokenizer, text_encoder)
X_val_text_feat = get_text_features(X_val_text, tokenizer, text_encoder)
X_test_text_feat = get_text_features(X_test_text, tokenizer, text_encoder)

# ------------------------------------------------------------
# Training
# ------------------------------------------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    [X_train_text_feat, X_train_image], y_train,
    validation_data=([X_val_text_feat, X_val_image], y_val),
    epochs=10,
    batch_size=64,
    callbacks=[early_stop]
)

# ------------------------------------------------------------
# Report
# ------------------------------------------------------------
predictions_prob = model.predict([X_test_text_feat, X_test_image])
predictions = np.argmax(predictions_prob, axis=1)
true_labels = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(true_labels, predictions, digits=4))

# ------------------------------------------------------------
# Timer End
end_time = time.time()
print(f"TinyCLIP+MobileNetV2- EARLY Fusion：{end_time - start_time:.4f} 秒")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is pro

Epoch 1/10
61/61 [==============================] - 20s 281ms/step - loss: 3.8199 - accuracy: 0.3664 - val_loss: 3.5140 - val_accuracy: 0.4559
Epoch 2/10
61/61 [==============================] - 15s 241ms/step - loss: 3.5921 - accuracy: 0.4496 - val_loss: 3.4675 - val_accuracy: 0.4764
Epoch 3/10
61/61 [==============================] - 15s 252ms/step - loss: 3.5301 - accuracy: 0.4680 - val_loss: 3.4283 - val_accuracy: 0.4805
Epoch 4/10
61/61 [==============================] - 15s 255ms/step - loss: 3.4896 - accuracy: 0.4770 - val_loss: 3.3948 - val_accuracy: 0.4743
Epoch 5/10
61/61 [==============================] - 17s 276ms/step - loss: 3.4641 - accuracy: 0.4760 - val_loss: 3.3653 - val_accuracy: 0.4825
Epoch 6/10
61/61 [==============================] - 16s 261ms/step - loss: 3.4206 - accuracy: 0.4816 - val_loss: 3.3382 - val_accuracy: 0.4990
Epoch 7/10
61/61 [==============================] - 16s 257ms/step - loss: 3.3670 - accuracy: 0.5030 - val_loss: 3.3107 - val_accuracy: 0.4949

# 2. LATE FUSION TinyCLIP

In [7]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from transformers import AutoTokenizer, TFCLIPModel
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

from transformers import CLIPProcessor, CLIPModel

# ------------------------------------------------------------
# Timer Start
start_time = time.time()

# ------------------------------------------------------------
# TinyCLIP/CLIP
# ------------------------------------------------------------

#text_model_name = "openai/clip-vit-base-patch32"  # 若有 TinyCLIP 的 TF 版本可替換
text_model_name = "wkcn/TinyCLIP-ViT-61M-32-Text-29M-LAION400M"

processor = CLIPProcessor.from_pretrained(text_model_name)
text_encoder = CLIPModel.from_pretrained(text_model_name)

tokenizer = AutoTokenizer.from_pretrained(text_model_name)
text_encoder = TFCLIPModel.from_pretrained(text_model_name)
# TFCLIPModel ->input_ids, Not necessarily needs attention_mask
text_encoder.trainable = False
def get_text_features(texts, processor, model, maxlen=77):
    encodings = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=maxlen,
        return_tensors="tf"
    )
    # TinyCLIP/CLIP TF -> get_text_features
    features = model.get_text_features(
        input_ids=encodings["input_ids"]
    )
    return features.numpy()

# Transfer to [str]
X_train_text = [str(t) for t in X_train_text]
X_val_text = [str(t) for t in X_val_text]
X_test_text = [str(t) for t in X_test_text]

#Extract FIRST : text data into numerical features
X_train_text_feat = get_text_features(X_train_text, tokenizer, text_encoder)
X_val_text_feat = get_text_features(X_val_text, tokenizer, text_encoder)
X_test_text_feat = get_text_features(X_test_text, tokenizer, text_encoder)

# ------------------------------------------------------------
# MobileNetV2 
# ------------------------------------------------------------
mobilenet = tf.keras.applications.MobileNetV2(include_top=False, 
                                              weights='imagenet', 
                                              input_shape=(imgsize, imgsize, 3))
mobilenet.trainable = False
image_input = Input(shape=(imgsize, imgsize, 3), name='image_input')

image_features = mobilenet(image_input)
image_features = GlobalAveragePooling2D()(image_features)
image_features = Dense(64, activation='relu')(image_features)

# ------------------------------------------------------------
# Fusion
# ------------------------------------------------------------
text_input = Input(shape=(X_train_text_feat.shape[1],), name='text_feat_input')
fusion = Concatenate()([text_input, image_features])
fusion = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(fusion)
fusion = Dropout(0.5)(fusion)  # 
output = Dense(3, activation='softmax')(fusion)

model = Model(inputs=[text_input, image_input], outputs=output)
opti = tf.keras.optimizers.AdamW(learning_rate=1e-4, clipnorm=1.0)
model.compile(loss='categorical_crossentropy', optimizer=opti, metrics=['accuracy'])
#model.summary()

# ------------------------------------------------------------
# Training
# ------------------------------------------------------------
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    [X_train_text_feat, X_train_image], y_train,
    validation_data=([X_val_text_feat, X_val_image], y_val),
    epochs=10,
    batch_size=64,
    callbacks=[early_stop]
)

# ------------------------------------------------------------
# Report
# ------------------------------------------------------------
predictions_prob = model.predict([X_test_text_feat, X_test_image])
predictions = np.argmax(predictions_prob, axis=1)
true_labels = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(true_labels, predictions, digits=4))

# ------------------------------------------------------------
# Timer End
end_time = time.time()
print(f"TinyCLIP+MobileNetV2-LATE Fusion：{end_time - start_time:.4f} 秒")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is pro

Epoch 1/10
61/61 [==============================] - 26s 359ms/step - loss: 2.1419 - accuracy: 0.4750 - val_loss: 2.0038 - val_accuracy: 0.5503
Epoch 2/10
61/61 [==============================] - 20s 324ms/step - loss: 1.9444 - accuracy: 0.5358 - val_loss: 1.8621 - val_accuracy: 0.5503
Epoch 3/10
61/61 [==============================] - 19s 315ms/step - loss: 1.7969 - accuracy: 0.5497 - val_loss: 1.7420 - val_accuracy: 0.5503
Epoch 4/10
61/61 [==============================] - 19s 319ms/step - loss: 1.6764 - accuracy: 0.5535 - val_loss: 1.6416 - val_accuracy: 0.5503
Epoch 5/10
61/61 [==============================] - 20s 321ms/step - loss: 1.5808 - accuracy: 0.5653 - val_loss: 1.5562 - val_accuracy: 0.5503
Epoch 6/10
61/61 [==============================] - 20s 334ms/step - loss: 1.4878 - accuracy: 0.5756 - val_loss: 1.4857 - val_accuracy: 0.5544
Epoch 7/10
61/61 [==============================] - 23s 377ms/step - loss: 1.4163 - accuracy: 0.5815 - val_loss: 1.4225 - val_accuracy: 0.5606